In [2]:
import sys
sys.path.append("../")
from utils import *

"""
Calculate total and average fuel consumption for a given vessel and time range.
"""
def calculate_fuel_consumption(vessel_id, start_time, end_time):
    # Define the parameter ID for fuel consumption
    parameter_ids = ["enginemain_fuelcons_lph_1"]

    try:
        # Fetch data from PONTOS-HUB
        vessel_data = fetch_vessel_data(
            vessel_id = vessel_id,
            start_time = start_time,
            end_time = end_time,
            parameter_ids = parameter_ids,
            time_bucket = "1 minute"
        )

        # Convert fetched data into a Pandas DataFrame
        fuel_data_df = transform_vessel_data_to_dataframe(vessel_data)

        if fuel_data_df.empty:
            return {"message": "No data available for the specified parameters."}

        # Filter out non-zero fuel consumption
        non_zero_data = fuel_data_df[fuel_data_df["enginemain_fuelcons_lph_1"] > 0]

        if non_zero_data.empty:
            return {"message": "No non-zero data available for the specified parameters."}

        # Calculate the total and average fuel consumption
        total_fuel_consumption = non_zero_data["enginemain_fuelcons_lph_1"].sum().round(2)
        average_fuel_consumption = non_zero_data["enginemain_fuelcons_lph_1"].mean().round(2)

        return {
            "vessel_id": vessel_id,
            "time_frame": f"{start_time} to {end_time}",
            "total_fuel_consumption": total_fuel_consumption,
            "average_fuel_consumption": average_fuel_consumption,
        }
    except Exception as e:
        return {"message": f"An error occurred: {str(e)}"}

# Example usage
vessel_id = "mmsi_265558290"
start_time = "2024-11-01 00:00:00"
end_time = "2024-11-01 23:59:59"

result = calculate_fuel_consumption(vessel_id, start_time, end_time)
if "message" in result:
    print(result["message"])
else:
    print(f"Vessel ID: {result['vessel_id']}")
    print(f"Time Frame: {result['time_frame']}")
    print(f"Total Fuel Consumption: {result['total_fuel_consumption']} liters")
    print(f"Average Fuel Consumption per minute: {result['average_fuel_consumption']} liters")

Vessel ID: mmsi_265558290
Time Frame: 2024-11-01 00:00:00 to 2024-11-01 23:59:59
Total Fuel Consumption: 8256.23 liters
Average Fuel Consumption per minute: 10.36 liters


In [ ]:
"""
Calculate total and average CO2 emissions for a specific vessel and time range using an emission factor based on the type of fuel.
"""
def calculate_co2_emissions_with_factor(vessel_id, start_time, end_time, emission_factor):
    try:
        # Fetch fuel consumption data using the helper function
        fuel_data = calculate_fuel_consumption(vessel_id, start_time, end_time)

        # Check if the fuel data has any issues
        if "message" in fuel_data:
            return {"message": fuel_data["message"]}

        # Calculate CO2 emissions based on total fuel consumption
        total_co2_emissions = (fuel_data["total_fuel_consumption"] * emission_factor).round(2)
        average_co2_emissions = (fuel_data["average_fuel_consumption"] * emission_factor).round(2)

        return {
            "vessel_id": vessel_id,
            "time_frame": fuel_data["time_frame"],
            "emission_factor": emission_factor,
            "total_co2_emissions": total_co2_emissions,
            "average_co2_emissions_per_minute": average_co2_emissions
        }
    except Exception as e:
        return {"message": f"An error occurred: {str(e)}"}

# Example usage
vessel_id = "mmsi_265558290"
start_time = "2024-11-01 00:00:00"
end_time = "2024-11-01 23:59:59"
emission_factor = 2.68  # Diesel emission factor (kg CO2 per liter)

result = calculate_co2_emissions_with_factor(vessel_id, start_time, end_time, emission_factor)

if "message" in result:
    print(result["message"])
else:
    print(f"Vessel ID: {result['vessel_id']}")
    print(f"Time Frame: {result['time_frame']}")
    print(f"Emission Factor: {result['emission_factor']} kg CO2/L")
    print(f"Total CO2 Emissions: {result['total_co2_emissions']} kg")
    print(f"Average CO2 Emissions per minute: {result['average_co2_emissions_per_minute']} kg")

Vessel ID: mmsi_265558290
Time Frame: 2024-11-01 00:00:00 to 2024-11-01 23:59:59
Emission Factor: 2.68 kg CO2/L
Total CO2 Emissions: 22126.7 kg
Average CO2 Emissions per minute: 27.76 kg
